In [1]:
import pandas as pd
import numpy as np
import plotly
from plotly import graph_objects as go

from model import DataManager

In [2]:
mx_data = pd.read_csv('data/mx_6-7.csv')

In [3]:
m = DataManager(mx_data, market='mx')

Clean data spans 2020-09 to 2022-02
Total # of cohorts: 18



In [4]:
m.plot_cohorts('default_rate_7dpd', 'backtest')

Data has not been forecast or backtested yet.
Run forecast_data() or backtest_data() methods first.
